WRITEUP OF IMPORTANT COLUMNS IN CSV

1. expStart - Provides a timestamp for when experiment started. EX: (2025-02-26 16h15.46.372660 -0800)

Each row below provides time in seconds about when each event occurs relative to start of experiment. EX (11.3732228)
1. aligning_image.started	polygon.started	polygon.stopped	aligning_image.stopped 
    - 3 rows of this, this describes when the big red clench jaw happens
    - Participant 2 did not have this. (administrative error)
2. mugshots.started
    - 5 rows of this, each is presentation of new face
    - In each row: new_face.key_resp.keys has what key is pressed
        - f corresponds to a perceived culprit
        - k corresponds to a perceived innocent
    - In each row: culprit is true or false depending on whether person scored
3. button_refresher.stopped and break_2.started
    - the refresher stop marks the start of a focus block
    - break start marks the end. 
    - Cleanest data is within these blocks

---

What we will do:
1. Read PsychoPy CSV
2. Create event code files for each participant
    - 7 different events: 1: TP, 2: FN, 3: FP, 4: TN, 5: clench jaw, 6: focus block start, 7: focus block end
    - Assign them "time" in terms of samples or data points. 
3. Read in Cyton, and fully align the data with sample points (was the recording done 1 minute in, or 10?)
    - Read the amount of samples that occurred before recording began, and add that value to entire event code


In [3]:


import pandas as pd
import numpy as np
import mne

from datetime import datetime

In [4]:
# https://openbci.com/forum/index.php?p=/discussion/3625/recorded-data-format-and-processing
headers = [
    "Sample Index", "EXG Channel 0", "EXG Channel 1", "EXG Channel 2", "EXG Channel 3", "EXG Channel 4", "EXG Channel 5", "EXG Channel 6", "EXG Channel 7", "Accel Channel 0", "Accel Channel 1", "Accel Channel 2", "Not Used 1", "Digital Channel 0", "Digital Channel 1", "Digital Channel 2", "Digital Channel 3", "Not Used 2", "Digital Channel 4", "Analog Channel 0", "Analog Channel 1", "Analog Channel 2", "Timestamp", "Marker Channel", "Timestamp (Formatted)"
]

In [40]:
data_dir = "Processed"
p_num = 5

In [41]:
with open(f'./Participant{p_num}/Participant{p_num}.txt', 'r') as f:
  while f.readline()[0] == "%":
    pass
  Cyton_DF = pd.read_csv(f, sep=",", names=headers, nrows=5) # not interested in whole file right now

In [42]:
Psycho_DF = pd.read_csv(f'./Participant{p_num}/Participant{p_num}.csv')
print(Psycho_DF.shape)
Psycho_DF.head()

(206, 91)


,thisN,thisTrialN,thisRepN,stim_vids,start_index,end_index,repetitions,stim_faces,culprit,key_resp.keys,...,new_clip.key_resp_2.rt,new_clip.key_resp_2.duration,participant,session,date,expName,psychopyVersion,frameRate,expStart,Unnamed: 90
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,kk,1,2025-04-10_20h09.18.842,MyProcedure,2024.2.4,59.976561,2025-04-10 20h09.34.689761 -0700,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,kk,1,2025-04-10_20h09.18.842,MyProcedure,2024.2.4,59.976561,2025-04-10 20h09.34.689761 -0700,NaN
2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,kk,1,2025-04-10_20h09.18.842,MyProcedure,2024.2.4,59.976561,2025-04-10 20h09.34.689761 -0700,NaN
3,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,kk,1,2025-04-10_20h09.18.842,MyProcedure,2024.2.4,59.976561,2025-04-10 20h09.34.689761 -0700,NaN
4,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,kk,1,2025-04-10_20h09.18.842,MyProcedure,2024.2.4,59.976561,2025-04-10 20h09.34.689761 -0700,NaN


In [43]:
psychoStart = Psycho_DF["expStart"][0]
print(psychoStart)

psychoStart = psychoStart.replace("h", ":")
dt = datetime.strptime(psychoStart, "%Y-%m-%d %H:%M.%S.%f %z")

# Convert to Unix timestamp (seconds)
psycho_unix = dt.timestamp()
print(f"Psycho Start in Unix Time: {psycho_unix}")

# Check cyton start
cyton_start = Cyton_DF["Timestamp"][0]
print(f"Cyton Start in Unix Time: {cyton_start}")

seconds_before_psycho = psycho_unix - cyton_start
# print(seconds_before_psycho)
samples_before_psycho = round(seconds_before_psycho * 250)

print(f"Recorded for {samples_before_psycho} samples before Psycho began")
print(f"Recorded for {(samples_before_psycho)/(60 * 250)} minutes before Psycho began")

2025-04-10 20h09.34.689761 -0700
Psycho Start in Unix Time: 1744340974.689761
Cyton Start in Unix Time: 1744340808.7725172
Recorded for 41479 samples before Psycho began
Recorded for 2.7652666666666668 minutes before Psycho began


In [44]:
polygon_present = "polygon.started" in Psycho_DF.columns # Checksafe for reasons I explain above

events = []

for index, event_row in Psycho_DF.iterrows():
  sample_num = samples_before_psycho  
  if polygon_present and pd.notna(event_row["polygon.started"]):
    sample_num += round(event_row["polygon.started"] * 250)
    events.append([sample_num, 0, 5])
    
  elif pd.notna(event_row["mugshots.started"]): # They should be mutually exclusive, but just making sure 
    if pd.notna(event_row["showing_movie.started"]):
      sample_num += round(event_row["button_refresher.stopped"] * 250)
      events.append([sample_num, 0, 6]) 
      sample_num = samples_before_psycho  

    key = event_row["key_resp.keys"]
    if pd.isna(key): # if no key entered
      continue
    flagged = (key == "f")
    culprit = event_row["culprit"]

    if culprit and flagged:  # True Positive
      category = 1
    elif culprit and not flagged:  # False Negative
      category = 2
    elif not culprit and flagged:  # False Positive
      category = 3
    else:  # not culprit and not flagged (True Negative)
      category = 4

    sample_num += round(event_row["mugshots.started"] * 250)
    events.append([sample_num, 0, category])

  elif pd.notna(event_row["break_2.started"]):
    sample_num += round(event_row["break_2.started"] * 250)
    events.append([sample_num, 0, 7]) 


events = np.array(events)
print(events)
print("------------------")
print(f"TP: {len([event for event in events if event[2]==1]):3} | FP: {len([event for event in events if event[2]==2]):3}")
print(f"FN: {len([event for event in events if event[2]==3]):3} | TN: {len([event for event in events if event[2]==4]):3}")

[[ 70605      0      5]
 [ 71480      0      5]
 [ 72355      0      5]
 [ 97788      0      6]
 [ 98792      0      1]
 [100292      0      4]
 [101792      0      4]
 [103292      0      4]
 [104792      0      4]
 [108167      0      4]
 [109667      0      1]
 [111167      0      4]
 [112667      0      4]
 [114167      0      4]
 [115039      0      7]
 [123251      0      6]
 [124255      0      4]
 [125755      0      4]
 [127255      0      4]
 [128755      0      4]
 [130255      0      1]
 [133630      0      4]
 [135130      0      1]
 [136630      0      4]
 [138130      0      4]
 [139630      0      4]
 [143005      0      4]
 [144505      0      4]
 [146005      0      1]
 [147505      0      4]
 [149005      0      4]
 [152380      0      4]
 [153880      0      4]
 [155380      0      4]
 [156880      0      1]
 [158380      0      4]
 [161755      0      4]
 [163255      0      1]
 [164755      0      4]
 [166255      0      4]
 [167755      0      4]
 [171130      0 

In [45]:

mne.write_events(f"{data_dir}/P{p_num}/P{p_num}_eve.fif", events, overwrite=True)

Overwriting existing file.
